# Introducing promptfoo

**Note: このレッスンは関連コードファイルを含むフォルダ内にあります。実際に手元で実行しながら進めたい場合は、フォルダ全体をダウンロードしてください。**

これまで、評価をゼロから自作する方法を見てきました。これは有効ですが、少し面倒でもあります。実務では、この目的のために設計された専用ツールを使う方が現実的なことが多いです。現在は評価ツール／ライブラリが多数あり（今も増え続けています）、例えば次のようなものがあります：
- [promptfoo](https://github.com/promptfoo/promptfoo)
- [Vellum](https://www.vellum.ai/#playground)
- [Scale Evaluation](https://scale.com/evaluation/model-developers)
- [Prompt Layer](https://promptlayer.com/)
- [Chain Forge](https://github.com/ianarawjo/ChainForge)
- ほか多数！

オープンソースで扱いやすい選択肢の一つが promptfoo です。promptfoo は、包括的なプロンプトテストに必要な時間と労力を大幅に減らせる、すぐ使える仕組みを提供します。バッチテスト、バージョン管理、性能分析のためのインフラが最初から用意されているので、開発者はテスト基盤を作るよりもプロンプトの改善に集中できます。複数のプロンプト・モデル・プロバイダにまたがる評価を簡単に実行でき、結果の可視化や比較もしやすいです。promptfoo などの評価ツールは、評価ロジックを自力で一から書くよりも大きな改善になります。

評価を実行すると、promptfoo は次の画像のようなダッシュボードも生成します：

![prompt_foo.png](../images/prompt_foo.png)

では始めましょう！

---

## Our first promptfoo eval

このコースの次の数レッスンでは、promptfoo を使った評価の書き方に焦点を当てます。この最初のレッスンでは、数レッスン前に扱った「この動物の脚は何本？」プロンプトを、promptfoo で評価するシンプルな方法を学びます。プロンプトも評価もかなり単純で、ここではツールの使い方と評価を回すプロセス自体に集中します。

復習として、そのレッスンでは次の小さな評価データセットを使いました：

```py
eval_data = [
    {"animal_statement": "The animal is a human.", "golden_answer": "2"},
    {"animal_statement": "The animal is a snake.", "golden_answer": "0"},
    {"animal_statement": "The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.", "golden_answer": "5"},
    {"animal_statement": "The animal is a dog.", "golden_answer": "4"},
    {"animal_statement": "The animal is a cat with two extra legs.", "golden_answer": "6"},
    {"animal_statement": "The animal is an elephant.", "golden_answer": "4"},
    {"animal_statement": "The animal is a bird.", "golden_answer": "2"},
    {"animal_statement": "The animal is a fish.", "golden_answer": "0"},
    {"animal_statement": "The animal is a spider with two extra legs", "golden_answer": "10"},
    {"animal_statement": "The animal is an octopus.", "golden_answer": "8"},
    {"animal_statement": "The animal is an octopus that lost two legs and then regrew three legs.", "golden_answer": "9"},
    {"animal_statement": "The animal is a two-headed, eight-legged mythical creature.", "golden_answer": "8"},
]
```

そのレッスンでは、素朴な評価関数を使って、精度が段階的に改善する3種類のプロンプトを書きました。このレッスンでは、その評価データセットとプロンプトを promptfoo に移植し、出力の実行と比較がどれだけ簡単かを確認します。


---

## Installing promptfoo

promptfoo を使う最初のステップは、コマンドラインからインストールすることです。評価コードを書くフォルダへ移動し、次のコマンドを実行します：

```bash
npx promptfoo@latest init
```

これにより、カレントディレクトリに `promptfooconfig.yaml` が作成されます。このファイルが設定の中心です。ここで次を設定します：
- 評価に使うプロバイダ（＝どの Anthropic API モデルを使うか）
- 評価したいプロンプト
- 実行したいテスト

---


## Configuring the provider(s)

次に、評価に使う Anthropic API モデルを promptfoo に設定します。`promptfooconfig.yaml` に `providers` フィールドを追加し、1つ以上の Anthropic モデルを指定します。promptfoo はモデル名指定に特定の形式を使います。現在サポートされている Anthropic の指定文字列は次のとおりです：

- `anthropic:messages:claude-3-5-sonnet-20240620`
- `anthropic:messages:claude-3-haiku-20240307`
- `anthropic:messages:claude-3-sonnet-20240229`
- `anthropic:messages:claude-3-opus-20240229`
- `anthropic:messages:claude-2.0`
- `anthropic:messages:claude-2.1`
- `anthropic:messages:claude-instant-1.2`

この最初の評価では Haiku を使います。`promptfooconfig.yaml` の既存内容を削除して、次の内容に置き換えてください：

```yaml
description: "Animal Legs Eval"
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"
```

各項目の意味：

- `description`：評価タスクの任意ラベル
- `providers`：この評価で Haiku を使うことを指定（将来のレッスンで複数モデル指定も行います）


promptfoo は評価実行時に `ANTHROPIC_API_KEY` 環境変数を探します。次のようにコマンドラインで環境変数を設定できます：

```bash
export ANTHROPIC_API_KEY=your_api_key_here
```

---


## Specifying our prompts

次は、評価したいプロンプトを promptfoo に教えます。方法はいくつかあります：
- YAMLファイルにプロンプト本文を直接書く
- JSONファイルから読み込む
- テキストファイルから読み込む
- 別のYAMLファイルから読み込む
- Pythonファイルから読み込む

ここでは、関連プロンプトを1つの Python ファイルにまとめ、プロンプト文字列を返す関数として定義する方法を使います。後のレッスンでは他の方法も見ます。promptfoo はかなり柔軟で、このコースを通してその柔軟性が分かるはずです。


`prompts.py` という名前の Python ファイルを作成し、次のプロンプト関数をその中に配置してください：

```py
def simple_prompt(animal_statement):
    return f"""動物に関する記述が提供されます。あなたの仕事は、その動物の脚の本数を判断することです。
    
    以下が動物に関する記述です。
    <animal_statement>{animal_statement}</animal_statement>
    
    この動物の脚は何本ですか？数字で回答してください。"""

def better_prompt(animal_statement):
    return f"""動物に関する記述が提供されます。あなたの仕事は、その動物の脚の本数を判断することです。
    
    以下が動物に関する記述です。
    <animal_statement>{animal_statement}</animal_statement>
    
    この動物の脚は何本ですか？2や9のような1桁の数字のみで回答してください。"""
```

それぞれの関数は `animal_statement` を受け取り、プロンプトに差し込んで最終的なプロンプト文字列を返します。


次に、作成した `prompts.py` からプロンプトを読み込むよう promptfoo の設定を更新します。`promptfooconfig.yaml` を次のようにしてください：

```yaml
description: "Animal Legs Eval"

prompts:
  - prompts.py:simple_prompt
  - prompts.py:better_prompt
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"
```

`prompts.py` に追加した各関数に対して、1行ずつ追加している点に注目してください。これで promptfoo は `simple_prompt` と `better_prompt` の両方を評価対象として扱います。

---


## Configuring our tests

次は、どのテストを実行するかを promptfoo に教えます。定義方法はいろいろありますが、ここでは最も一般的な方法の一つである **CSVファイル** を使います。

`dataset.csv` というCSVファイルを作り、そこにテスト入力を書いていきます。

promptfoo では、評価ロジック（アサーション）をCSV内で定義することもできます。今後のレッスンで組み込みアサーションも扱いますが、この評価では「モデル出力」と「期待値（脚の数）」の **完全一致** を見れば十分です。

そのため、CSVの列は次の2つを用意します：
- `animal_statement`：入力（例："The animal is an elephant"）
- `__expected`：期待される正しい出力（`__expected` の `__` は promptfoo 固有の構文です）


`dataset.csv` ファイルを作成し、次の内容を追加してください：

```csv
animal_statement,__expected
"The animal is a human.","2"
"The animal is a snake.","0"
"The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.","5"
"The animal is a dog.","4"
"The animal is a cat with two extra legs.","6"
"The animal is an elephant.","4"
"The animal is a bird.","2"
"The animal is a fish.","0"
"The animal is a spider with two extra legs","10"
"The animal is an octopus.","8"
"The animal is an octopus that lost two legs and then regrew three legs.","9"
"The animal is a two-headed, eight-legged mythical creature.","8"
```


最後に、promptfoo が `dataset.csv` からテストを読み込むよう設定します。`promptfooconfig.yaml` に次を追加（または更新）してください：

```yaml
description: "Animal Legs Eval"

prompts:
  - prompts.py:simple_prompt
  - prompts.py:better_prompt
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"

tests: animal_legs_tests.csv
```

---


## Running our evaluation

プロバイダ、プロンプト、テストが揃ったので、評価を実行しましょう。

ターミナルで次のコマンドを実行します：

```bash
npx promptfoo@latest eval
```

これで評価が開始されます。各プロンプトについて、promptfoo は次を行います：
- CSVから各 `animal_statement` を取り出す
- `animal_statement` を含む完成プロンプトを組み立てる
- 個別プロンプトを Anthropic API に送る
- 出力がCSVの期待値と一致するかチェックする

評価が終わると、promptfoo はターミナルに結果を表示します。


これは、上の設定で実行した promptfoo のターミナル出力例です：

![eval_output1.png](../images/eval_output1.png)


上のスクリーンショットは最初の4行しか映っていませんが、評価は12件すべてで実行されています。
- 左列：`animal_statement`
- 中列：`simple_prompt` の出力とスコア（全行で失敗していそうです）
- 右列：`better_prompt` の出力とスコア（多くは成功するが、論理的に複雑な入力で失敗する）

---


## Viewing the evaluation

promptfoo は評価結果をブラウザで可視化・調査できるダッシュボードを簡単に立ち上げられます。上の eval を実行した後、ターミナルで次を実行してください：

```bash
npx promptfoo@latest view
```

サーバーを開始するか聞かれるので（'y' を入力）、ブラウザでダッシュボードが開きます。

![eval_view1.png](../images/eval_view1.png)


重要なサマリー情報は画面上部にあります：

![eval_results1.png](../images/eval_results1.png)


失敗した理由を理解するために、特定の結果に注目することもできます。`simple_prompt` の結果（中央列）を見てみましょう。全行が失敗になっています。なぜでしょう？

セル内の虫眼鏡ボタンをクリックすると詳細を確認できます：

![toolbar.png](../images/toolbar.png)


出力と採点の詳細が載ったモーダルが開きます：

![details.png](../images/details.png)


この例では `simple_prompt` は答え自体（0）は正しいのに、余計な説明文が大量に含まれているため、完全一致の評価で失敗しています。

一方、右端列（`better_prompt`）を見ると、`5` や `0` のように1桁の数値だけを返すケースが多く、かなり改善しています。ただし、次のような推論が必要な "複雑な" `animal_statement` では失敗しているようです：

> The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.

---


## Adding in a third prompt

コード採点評価のレッスンでは、最終的に chain-of-thought（思考の連鎖）を入れることで最も良い結果になりました。同様に、chain-of-thought を含む第3の改善プロンプトを追加し、「ひっかけ」入力での性能を見てみましょう。

`prompts.py` に次の関数を追加してください：

```py
def chain_of_thought_prompt(animal_statement):
    return f"""動物に関する記述が提供されます。あなたの仕事は、その動物の脚の本数を判断することです。
    
    以下が動物に関する記述です。
    <animal_statement>{animal_statement}</animal_statement>
    
    この動物の脚は何本ですか？ 
    まず、<thinking>タグ内で段階的に思考し、動物の脚の本数について推論してください。  
    その後、<answer>タグ内に最終的な答えを出力してください。 
    <answer>タグ内には、脚の本数を整数のみで返し、それ以外は何も含めないでください。"""
```


次に、新しいプロンプトを `promptfooconfig.yaml` に追加します：

```yaml
description: "Animal Legs Eval"

prompts:
  - prompts.py:simple_prompt
  - prompts.py:better_prompt
  - prompts.py:chain_of_thought_prompt
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"

tests: animal_legs_tests.csv
```


このままでは1つ問題があります。新しい `chain_of_thought_prompt` は出力に `<thinking>` と `<answer>` タグを含みます。このプロンプトの性能を正しく評価するには、`<answer>` タグ内の数値だけを抽出し、それを期待値と比較する必要があります。

promptfoo には、比較前にモデル出力を加工できるカスタム `transforms` が用意されています。ここでは `<answer>` の中身を取り出す簡単な Python 関数を書きます。

`transform.py` を新規作成し、次を追加してください：

```py
def get_transform(output, context):
    if "<thinking>" in output:
        try:
            return output.split("<answer>")[1].split("</answer>")[0].strip()
        except Exception as e:
            print(f"Error in get_transform: {e}")
            return output
    return output
```

この `get_transform` 関数はモデル出力（`output`）を受け取り（`context` については後のレッスンで扱います）、比較前に好きな形へ変換して返せます。ここでは次の2通りの処理をします：

- 出力に `<thinking>` が含まれる場合（chain-of-thought の出力と判断）：`<answer>` の中の数値を抽出して返す
- それ以外：元の出力をそのまま返す（chain-of-thought を使わない他のプロンプト向け）


最後に、この変換関数を使うよう promptfoo に指示します。`promptfooconfig.yaml` を次のように更新してください：

```yaml
description: "Animal Legs Eval"

prompts:
  - prompts.py:simple_prompt
  - prompts.py:better_prompt
  - prompts.py:chain_of_thought_prompt
  
providers:
  - "anthropic:messages:claude-3-haiku-20240307"

tests: animal_legs_tests.csv

defaultTest:
  options:
    transform: file://transform.py
```


末尾の指定により、promptfoo は全テストに対して `transform.py` の変換関数を常に適用します。デフォルトでは、`transform.py` 内の `get_transform` という関数を探します。


これで、次のコマンドで評価を再実行できます：

```bash
npx promptfoo@latest eval
```


次のように、今度は4列になった出力が表示されるはずです：

![three_prompt_eval.png](../images/three_prompt_eval.png)


ブラウザで結果を見るには、再び次を実行します：

```bash
npx promptfoo@latest view
```

次のようなページが表示されます：

![final_view.png](../images/final_view.png)


chain-of-thought を含むプロンプトが、全テストで 100% 正解していることが分かります！

---


## Comparing models

promptfoo の便利な点の一つは、異なるモデルで評価を回すのが非常に簡単なことです。Haiku で 100% を出すためにプロンプトを工夫しましたが、より高性能な Claude 3.5 Sonnet に切り替えたらどうなるでしょう？

必要なのは、`promptfooconfig.yaml` の `providers` に2つ目のプロバイダを追加するだけです。次のように2つの providers を指定してください：

```yaml
description: "Animal Legs Eval"

prompts:
  - prompts.py:simple_prompt
  - prompts.py:better_prompt
  - prompts.py:chain_of_thought_prompt
  
providers:
  - anthropic:messages:claude-3-haiku-20240307
  - anthropic:messages:claude-3-5-sonnet-20240620

tests: animal_legs_tests.csv

defaultTest:
  options:
    transform: file://transform.py
```

そして、先ほどと同じコマンドで評価を再実行します：

```bash
npx promptfoo@latest eval
```


ブラウザのダッシュボードを見ると、興味深い結果が出ます：

![multi_model_eval_view.png](../images/multi_model_eval_view.png)


YAML にたった1行追加しただけで、2つのモデルにまたがって同じ評価セットを実行できました。最初の3つの出力列は Claude 3 Haiku、最後の3つは Claude 3.5 Sonnet の出力です。Claude 3 Haiku では `simple_prompt` が 0% だったのに、Claude 3.5 Sonnet では `simple_prompt` でも 100% で通っているように見えます。

この種の情報は非常に価値があります。「どのプロンプトが一番良いか」だけでなく、「どのモデル＋プロンプトの組み合わせがそのタスクで最も良いか」を判断できるからです。

**補足：** Claude 3.5 Sonnet が chain-of-thought のプロンプトで 100% になっていない理由が気になる場合、説明します。`animal_statement` が "The animal is an octopus." のテストで誤っており、`<thinking>` 内で「タコは脚（legs）ではなく腕（arms）と呼ばれる付属肢を持ち、脚とは言わない」という推論をしています。より賢いモデルに変えたことで、chain-of-thought プロンプトでは逆に“賢すぎる”推論により性能が少し落ちた、というケースです。もし全モデルでの性能を揃えたいなら、「何を leg とみなすか」をプロンプトでより明確にする、といった対策ができます。

このレッスンは promptfoo の最初の触りです。今後のレッスンでは、より複雑なコード採点、独自のカスタム採点器（grader）、モデル採点の評価などを学びます。
